https://github.com/prdeepakbabu/Python/blob/master/Deep%20learning%20gender/Deep%20Learning%20(RNN%20-%20LSTMs)%20Predict%20Gender%20from%20Name.ipynb 

https://github.com/mhjabreel/CharCnn_Keras/blob/master/models/char_cnn_zhang.py 

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from tensorflow.python.keras.layers.core import Dense, Activation, Dropout
from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.models import Sequential, Model, load_model
from tensorflow.python.keras.layers import Dense, Embedding
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers import Input, Dense, Flatten
from tensorflow.python.keras.callbacks import TensorBoard
import pandas as pd
import numpy as np
import os
from sklearn.metrics import classification_report

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
PATH_TO_FOLDER = "drive/MyDrive/Name2Demographics/"

In [5]:
%cd $PATH_TO_FOLDER

/content/drive/MyDrive/Name2Demographics


In [6]:
import sys  
sys.path.insert(0, 'Models/PreProcessing/')

from char_utils import *

sys.path.insert(0, 'Models/CBSEData/CBSEPreprocessing/')
from parse_cbse import *

In [7]:
# print(tf.__version__)

In [8]:
# physical_devices = tf.config.list_physical_devices('GPU')
# # tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
# from tensorflow.python.client import device_lib

# def get_available_devices():
#     local_device_protos = device_lib.list_local_devices()
#     return [x.name for x in local_device_protos if x.device_type == 'GPU' or x.device_type == 'CPU']

In [9]:
# get_available_devices()

### PreProcessing

In [10]:
cbse_df = CBSEData()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (84,88,90,94,96,100,102,106,108,112,114,118,120,123,125,128,130,134,136,140,142,145,147,151,153,157,159,163,165,169,171,175,177,180,182,185,187,191,193,194,196,200,202,206,208,211,213,216,218,222,224,228,230,234,236,240,242,246,248,252,254,257,259,263,265,269,271,275,277,281,283,287,289,293,295,299,301,305,307,311,313,317,319,323,325,328,330,334,336,340,342,346,348,351,353,354,356,359,361,364,366,370,372,376,378,381,383,387,389,393,395,399,401,405,407,411,413,417,419,423) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (82,86,88,92,94,98,100,104,106,110,112,116,124,128,130,134,136,140,142,146,148,152,154,158,160,164,166,170,172,176,178,182,184,188,190,194,196,200,202,206,208,212,214,218,220,224,226,230,232,233,235,239,241,245,247,2

In [11]:
train_x, train_y, val_x, val_y, test_x, test_y = splitData(cbse_df, 0.7, 0, 'Name', 'char_lstm_cbse_oov.txt')

(train:(139365, 4), val:(0, 0), test:(65179, 4))
Train: ((139324, 30),(139324, 2)) Val:((0,),(0,)) Test: ((65155, 30),(65155, 2))


In [12]:
voc = vocab()

### Build Model

In [ ]:
# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  

In [13]:
inputs = Input(MAXLEN_NAME_CHAR_LSTM)
x = Embedding(len(voc), len(voc))(inputs)
# x = LSTM(64, return_sequences=True, activation='tanh')(x)
# x = Dropout(0.2)(x)
x = LSTM(64, return_sequences=False, activation='tanh')(x)
x = Dropout(0.2)(x)
# x = Dense(8, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 30, 28)            784       
_________________________________________________________________
lstm (LSTM)                  (None, 64)                23808     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 2)                 130       
Total params: 24,722
Trainable params: 24,722
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [16]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [17]:
batch_size=100

In [ ]:
model.fit(train_x, train_y,batch_size=batch_size,epochs=10,validation_data=(test_x, test_y))

Epoch 1/10
1394/1394 [==============================] - 279s 197ms/step - loss: 0.4130 - accuracy: 0.8112 - val_loss: 0.3193 - val_accuracy: 0.8689
Epoch 2/10
1394/1394 [==============================] - 276s 198ms/step - loss: 0.2517 - accuracy: 0.9031 - val_loss: 0.2519 - val_accuracy: 0.9055
Epoch 3/10
1394/1394 [==============================] - 276s 198ms/step - loss: 0.2014 - accuracy: 0.9277 - val_loss: 0.2203 - val_accuracy: 0.9203
Epoch 4/10
1394/1394 [==============================] - 274s 196ms/step - loss: 0.1742 - accuracy: 0.9395 - val_loss: 0.2043 - val_accuracy: 0.9268
Epoch 5/10
1394/1394 [==============================] - 275s 197ms/step - loss: 0.1564 - accuracy: 0.9467 - val_loss: 0.2075 - val_accuracy: 0.9255
Epoch 6/10
1394/1394 [==============================] - 275s 198ms/step - loss: 0.1450 - accuracy: 0.9511 - val_loss: 0.2040 - val_accuracy: 0.9252
Epoch 7/10
1394/1394 [==============================] - 274s 197ms/step - loss: 0.1357 - accuracy: 0.9539 - val_

In [19]:
score, acc = model.evaluate(test_x, test_y)
print('Test score:', score)
print('Test accuracy:', acc)

2037/2037 [==============================] - 33s 16ms/step - loss: 0.2090 - accuracy: 0.9200
Test score: 0.20898273587226868
Test accuracy: 0.9200214743614197


In [20]:
pred = model.predict(np.asarray(test_x))
y_pred = np.argmax(pred, axis=1)
y_true = np.argmax(test_y, axis=1)

target_names = ['Male','Female']

print("------------------RESULTS-------------------------------")
print(classification_report(y_true, y_pred, target_names=target_names, digits=4))
print("--------------------------------------------------------")

------------------RESULTS-------------------------------
              precision    recall  f1-score   support

        Male     0.9277    0.9240    0.9258     35207
      Female     0.9111    0.9153    0.9132     29948

    accuracy                         0.9200     65155
   macro avg     0.9194    0.9197    0.9195     65155
weighted avg     0.9201    0.9200    0.9200     65155

--------------------------------------------------------


In [21]:
filename = "Models/CBSEData/CharLSTM/SavedModel/CharLSTM"
model.save(filename)

INFO:tensorflow:Assets written to: Models/CBSEData/CharLSTM/SavedModel/CharLSTM/assets
